In [687]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2,SequentialFeatureSelector as sfs

In [688]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [689]:
## setting work directory

In [690]:
os.chdir("U:\Propensity_Model\Model")

In [691]:
### Loading the required files

In [692]:
pickle_in = open("Final_Data_.pickle","rb")

Final_Data = pickle.load(pickle_in)

pickle_in.close()

In [693]:
### Split data into Train, Test and Validation

In [694]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [695]:
## Checking the Selection %

In [696]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #1.77
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #1.88

0    98.226685
1     1.773315
Name: AUDIT_STATUS, dtype: float64
0    98.119926
1     1.880074
Name: AUDIT_STATUS, dtype: float64


In [697]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [698]:
train_num = x_train.select_dtypes(include='number')

In [699]:
##PreProcessing numeric data

In [700]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [701]:
### Combining the processed data to train_data

In [702]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [703]:
## Combining the PreProcessed data to train_data

In [704]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [705]:
x_train = x_train.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [706]:
## performing one-hot-encoding on categorical features

In [707]:
ctgrl_col = pd.DataFrame(x_train[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

In [708]:
### Data balancing using SMOTE

In [709]:
sm = SMOTE(sampling_strategy=0.43)

In [710]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [576]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [577]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [578]:
y_sm.value_counts(normalize=True)*100

AUDIT_STATUS
0               69.931045
1               30.068955
dtype: float64

In [579]:
y_train.value_counts(normalize=True)*100

AUDIT_STATUS
0               98.226685
1                1.773315
dtype: float64

In [580]:
####################################################
####################################################
####################################################

### Model Building

In [535]:
XG_v1 = XGBClassifier()

In [536]:
data = {
    "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
}

In [537]:
from sklearn.model_selection import RandomizedSearchCV
gscv = RandomizedSearchCV(XG_v1,param_distributions=data,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)
gscv.fit(x_sm.values,y_sm.values.ravel(),eval_metric='error')

Fitting 5 folds for each of 5 candidates, totalling 25 fits


KeyboardInterrupt: 

In [155]:
gscv.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3,
              enable_categorical=False, gamma=0.2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=12,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [153]:
gscv.best_params_

{'min_child_weight': 7,
 'max_depth': 12,
 'learning_rate': 0.2,
 'gamma': 0.2,
 'colsample_bytree': 0.3}

In [154]:
gscv.best_score_

0.9947116335608056

In [711]:
XG_v1 = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3,
              enable_categorical=False, gamma=0.2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=12,
              min_child_weight=7, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)
XG_v1.fit(x_sm,y_sm,eval_metric='logloss')

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3,
              enable_categorical=False, gamma=0.2, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.2, max_delta_step=0, max_depth=12,
              min_child_weight=7, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [712]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [713]:
test_num = x_test.select_dtypes(include='number')

In [714]:
##PreProcessing numeric data

In [715]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [716]:
###########
### Combining the processed data to test_data

In [717]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [718]:
## Combining the Preprocessed data to test_data

In [719]:
x_test = pd.concat([x_test,test_num_process],axis=1)

In [720]:
x_test = x_test.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [721]:
##one-hot-encoding on categorical features

In [722]:
ctgrl_col = pd.DataFrame(x_test[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)
x_test=x_test.values

In [723]:
##################
### Predicting on the test data

In [724]:
##RF_v1

In [726]:
y_pred = XG_v1.predict(x_test)
y_pred = (XG_v1.predict_proba(x_test)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

0.9796325309784931
[[6871   18]
 [ 125    7]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6889
           1       0.28      0.05      0.09       132

    accuracy                           0.98      7021
   macro avg       0.63      0.53      0.54      7021
weighted avg       0.97      0.98      0.97      7021



In [499]:
y_pred = XG_v1.predict(x_sm)
y_pred = (XG_v1.predict_proba(x_sm)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

0.99257212391624
[[27575    10]
 [  283 11578]]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     27585
           1       1.00      0.98      0.99     11861

    accuracy                           0.99     39446
   macro avg       0.99      0.99      0.99     39446
weighted avg       0.99      0.99      0.99     39446

